In [1]:
import cv2
import os
import re
import numpy as np
from tqdm import tqdm

In [2]:
WIDTH = int(720)

def normilizeFrame(frame):
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    frame = cv2.resize(frame, (WIDTH, frame.shape[0]), interpolation=cv2.INTER_NEAREST)
    return frame

cap = cv2.VideoCapture('testfiles/999.avi')
if (cap.isOpened()== False): 
  print("Error opening video stream or file")

ret, frame = cap.read()
if not ret:
    print('error reading frame')

gray = normilizeFrame(frame)

In [3]:
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
frameCount = cap.get(cv2.CAP_PROP_FRAME_COUNT)

print(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
print(height)
print(cap.get(cv2.CAP_PROP_POS_FRAMES))
print(frameCount)

720.0
625.0
1.0
751.0


In [4]:
vl = gray[20]
vline = np.array([vl,vl,vl,vl,vl,vl,vl,vl,vl,vl,vl,vl,vl,vl,vl,vl,vl,vl,vl,vl,vl,vl,vl,vl,vl,vl,vl,vl,vl,vl,vl,vl,vl,vl,vl])

cv2.imwrite("gray.png", vline)

True

In [5]:
#fileIn = open("out_parse.log", "rt") 
fileInBin = open("out_bin.log", "rt") 

if os.path.exists("ignored.txt"):
  os.remove("ignored.txt")

fileIgnored = open("ignored.txt", "wt")

#line = fileIn.readline()
lineBin = fileInBin.readline()


In [6]:
#falses = np.zeros(16, dtype=bool)
dictionary = []  #[frame, line]
inputLines = []
service = [] #[has pcm]
binaryData = []
#blockL0 = []
#blockR0 = []
#blockL1 = []
#blockR1 = []
#blockL2 = []
#blockR2 = []

In [7]:
def convertDataBlock(data):
    data = list(data.replace('#', '1').replace('-', '0'))
    data = [bool(int(numeric_string)) for numeric_string in data]
    
    data = data[4:88]

    return bitsToInt(data[:14]), bitsToInt(data[14:28]), bitsToInt(data[28:42]), bitsToInt(data[42:56]), bitsToInt(data[56:70]), bitsToInt(data[70:84]) #L0, R0, L1, R1, L2, R2

bitsToInt = lambda ls: int("".join([str(int(b)) for b in ls]), 2)

bar = tqdm(total=height * frameCount)
bar.set_description("Parsing govnologs Phase 1")

Parsing govnologs Phase 1:   0%|                                                                                                                                                                               | 0/469375.0 [00:00<?, ?it/s]

In [9]:
while lineBin:
    typeLine=re.match('\"?\[(V2D)\]', lineBin)
    if typeLine != None:
        typeLine = typeLine.group(1)
        if typeLine == 'V2D':
            bar.update(1)
            parse = re.match('.?\[V2D\].*F\[(\d+)\].*L\[(\d+)\].*\[([#-]{137})\].*(CRC OK|No PCM!|BD CRC!)', lineBin)
            if parse and parse.group(3):
                framenum = int(parse.group(1))
                linenum = int(parse.group(2))
                currentCapPos = int(cap.get(cv2.CAP_PROP_POS_FRAMES))
                bar.update(1)
                
                #compare frame with log output
                if not framenum == currentCapPos:
                    if framenum > currentCapPos:
                        if cap.grab():
                            ret, frame = cap.retrieve()
                            
                            if not ret:
                                print('error reading frame')
                                break
                            
                            if not framenum == cap.get(cv2.CAP_PROP_POS_FRAMES):
                                print('WTF?! after read next frame number also not equals ' + str(framenum) + ' ' + str(currentCapPos))
                                break
                            
                            gray = normilizeFrame(frame)
                    else:
                        print('AHTUNG!!! ' + str(framenum) + ' ' + str(currentCapPos))
                        break
                
                dictionary.append([framenum, linenum])
                inputLines.append(gray[linenum - 1])
                binaryData.append(convertDataBlock(parse.group(3)))
               
                if parse.group(4) == 'No PCM!':
                    service.append([False])
                else:
                    service.append([True])
    lineBin = fileInBin.readline()
cap.release()
bar.close()

Parsing govnologs Phase 1: : 941635it [00:48, 19612.49it/s]                                                                                                                                                                                 


In [11]:
bar = tqdm(total=3)
bar.set_description("Save govnodata")

np.save('traindata/inputLines.npy', inputLines)
bar.update(1)

np.save('traindata/service.npy', service)
bar.update(1)

np.save('traindata/binaryData.npy', binaryData)
bar.update(1)

#np.save('traindata/blockL0.npy', blockL0)
#bar.update(1)
#
#np.save('traindata/blockR0.npy', blockR0)
#bar.update(1)
#
#np.save('traindata/blockL1.npy', blockL1)
#bar.update(1)
#
#np.save('traindata/blockR1.npy', blockR1)
#bar.update(1)
#
#np.save('traindata/blockL2.npy', blockL2)
#bar.update(1)
#
#np.save('traindata/blockR2.npy', blockR2)
#bar.update(1)
bar.close()


Save govnodata:  25%|██████████████████████████████████████████████▎                                                                                                                                          | 2/8 [00:18<00:56,  9.37s/it]

Save govnodata: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:01<00:00,  1.85it/s]


In [12]:
cap.release()
cv2.destroyAllWindows()